# Decision Trees

In [1]:
import pandas as pd
import numpy as np
# imports from sklearn
from sklearn import tree
from sklearn.model_selection import GridSearchCV, StratifiedKFold

dt = tree.DecisionTreeClassifier()

In [2]:
#loading in the datasets
#pca datasets
df_pca_training = pd.read_csv('../data/archive/train_pca.csv')
df_pca_testing = pd.read_csv('../data/archive/test_pca.csv')

#original datasets
df_training = pd.read_csv('../data/archive/train.csv')
df_testing = pd.read_csv('../data/archive/test.csv')

In [3]:
#defining x and y with pca dataset
x_pca_train = df_pca_training.iloc[:, :-1]
y_pca_train = df_pca_training.iloc[:,-1]

x_pca_test = df_pca_testing.iloc[:, :-1]
y_pca_test = df_pca_testing.iloc[:,-1]

In [4]:
#defining x and y with original dataset
x_train = df_training.iloc[:, :-2]
y_train = df_training.iloc[:,-1]

x_test = df_testing.iloc[:, :-2]
y_test = df_testing.iloc[:,-1]

In [5]:
#Making a grid of values we want our grid search to test to find the best parameters

grid_values = [{'criterion': ['gini', 'entropy'], 'max_depth' :list(range(1,104,4)), 'min_samples_leaf':list(range(10,100,10))}]

dt_classifier = GridSearchCV(estimator = dt, param_grid = grid_values, cv = StratifiedKFold(n_splits = 5),
                      scoring = ['accuracy', 'roc_auc_ovr', 'f1_micro'], refit = False, verbose = 0)

dt_pca_model = dt_classifier.fit(x_pca_train, y_pca_train)
dt_model = dt_classifier.fit(x_train, y_train)

In [6]:
#initializing our lists so we can later store the best params that achieve best scores across metrics
accuracy_best_params = []
precision_best_params = []
f1_best_params = []

def params(model):
    accuracy_best_params.append(model.cv_results_['params'][ np.argmin(model.cv_results_['rank_test_accuracy'])])
    precision_best_params.append(model.cv_results_['params'][ np.argmin(model.cv_results_['rank_test_roc_auc_ovr'])])
    f1_best_params.append(model.cv_results_['params'][ np.argmin(model.cv_results_['rank_test_f1_micro']) ])
    
    print(accuracy_best_params)
    print(precision_best_params)
    print(f1_best_params)

In [7]:
params(dt_model)

[{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 10}]
[{'criterion': 'gini', 'max_depth': 81, 'min_samples_leaf': 60}]
[{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 10}]


In [8]:
params(dt_pca_model)

[{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 10}, {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 10}]
[{'criterion': 'gini', 'max_depth': 81, 'min_samples_leaf': 60}, {'criterion': 'gini', 'max_depth': 81, 'min_samples_leaf': 60}]
[{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 10}, {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 10}]


In [13]:
#finding the accuracy metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score
    
def accuracy_metric(xtrain,ytrain,xtest,ytest):
    accuracy_test_score = []
    dt_clf = tree.DecisionTreeClassifier(criterion=accuracy_best_params[0]['criterion'], 
                                         max_depth = accuracy_best_params[0]['max_depth'], 
                                         min_samples_leaf = accuracy_best_params[0]['min_samples_leaf'])
    model = dt_clf.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    accuracy_test_score.append(accuracy_score(ytest, y_pred))
    return accuracy_test_score

def f1_metric(xtrain,ytrain,xtest,ytest):
    f1_test_score = []
    dt_clf = tree.DecisionTreeClassifier(criterion=f1_best_params[0]['criterion'], 
                                         max_depth = f1_best_params[0]['max_depth'], 
                                         min_samples_leaf = f1_best_params[0]['min_samples_leaf'])
    model = dt_clf.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    f1_test_score.append(f1_score(ytest, y_pred,average = 'micro'))
    return f1_test_score

def precision_metric(xtrain,ytrain,xtest,ytest):
    precision_test_score = []
    dt_clf = tree.DecisionTreeClassifier(criterion=precision_best_params[0]['criterion'], 
                                         max_depth = precision_best_params[0]['max_depth'], 
                                         min_samples_leaf = precision_best_params[0]['min_samples_leaf'])
    model = dt_clf.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    precision_test_score.append(precision_score(ytest, y_pred,average = 'micro'))
    return precision_test_score

In [14]:
print(accuracy_metric(x_train,y_train,x_test,y_test))
print(f1_metric(x_train,y_train,x_test,y_test))
print(precision_metric(x_train,y_train,x_test,y_test))

[0.8581608415337632]
[0.8608754665761792]
[0.8588394977943672]


In [15]:
print(accuracy_metric(x_pca_train,y_pca_train,x_pca_test,y_pca_test))
print(f1_metric(x_pca_train,y_pca_train,x_pca_test,y_pca_test))
print(precision_metric(x_pca_train,y_pca_train,x_pca_test,y_pca_test))

[0.8177807940278249]
[0.8177807940278249]
[0.7991177468612148]
